In [1]:
import json
import copy

FOLDER_DATA = "../../json_final/input"

count_risky_PP = 0

# load json with votes
try:
    with open(FOLDER_DATA + '/votes_dumped_2018_May_16.json') as data_file:
        data_votes_may_2 = json.load(data_file)
except:
    raise Exception(
        'ADD THE FILE CONTAINING THE VOTES (list of dict "value", "freesound_sound_id", "node_id") AND ADD IT TO THE FOLDER ' + FOLDER_DATA + 'json/')
try:
    with open(FOLDER_DATA + '/votes_dumped_2018_Jan_22.json') as data_file:
        data_votes_jan = json.load(data_file)
except:
    raise Exception(
        'ADD THE FILE CONTAINING THE VOTES (list of dict "value", "freesound_sound_id", "node_id") AND ADD IT TO THE FOLDER ' + FOLDER_DATA + 'json/')

try:
    with open(FOLDER_DATA + '/votes_dumped_2018_Mar_13.json') as data_file:
        data_votes_mar = json.load(data_file)
except:
    raise Exception(
        'ADD THE FILE CONTAINING THE VOTES (list of dict "value", "freesound_sound_id", "node_id") AND ADD IT TO THE FOLDER ' + FOLDER_DATA + 'json/')

try:
    with open(FOLDER_DATA + '/votes_dumped_2018_May_9.json') as data_file:
        data_votes_may_9 = json.load(data_file)
except:
    raise Exception(
        'ADD THE FILE CONTAINING THE VOTES (list of dict "value", "freesound_sound_id", "node_id") AND ADD IT TO THE FOLDER ' + FOLDER_DATA + 'json/')

# load json with ids and domestic categories info
try:
    with open(FOLDER_DATA + '/domestic_ids.json') as data_file:
        domestic_ids = json.load(data_file)
except:
    raise Exception('ADD AN ONTOLOGY JSON FILE TO THE FOLDER ' + FOLDER_DATA + 'json/')

In [2]:
def check_GT(group, fsid, catid, vote_groups, fsids_assigned_cat, data_sounds):
    # check if fsid has GT within a given group (PP,PNP,NP,U) of a category given by catid
    # if it does, add it to assigned fsids and send it to the corresponding group in data_sounds
    assigned = False
    if vote_groups[group].count(fsid) > 1:
        data_sounds[catid][group].append(fsid)
        fsids_assigned_cat.append(fsid)
        assigned = True
    return data_sounds, fsids_assigned_cat, assigned


def map_votedsound_2_disjointgroups_wo_agreement(fsid, catid, vote_groups, fsids_assigned_cat, data_sounds,
                                                 error_mapping_count_cat, count_risky_PP):
    # map the voted sound to a disjoint group  without inter-annotator agreement
    # using set of arbitrary rules that cover all possible options
    # being demanding now. only sending to PP when we are sure

    # retrieve votes for fsid.
    # we know there is only one in PP
    votes = []

    if fsid in vote_groups['PP']:
        votes.append(1.0)
    if fsid in vote_groups['PNP']:
        votes.append(0.5)
    if fsid in vote_groups['U']:
        votes.append(0.0)
    if fsid in vote_groups['NP']:
        votes.append(-1.0)

    # votes has all the votes for fsid. let us take decisions

    # trivial cases where there is only one single vote by one annotator
    if 1.0 in votes and 0.5 not in votes and -1.0 not in votes and 0.0 not in votes:
        # the only case where a sound is sent to PP without inter-annotator agreement
        data_sounds[catid]['PP'].append(fsid)
        fsids_assigned_cat.append(fsid)
    elif 1.0 not in votes and 0.5 in votes and -1.0 not in votes and 0.0 not in votes:
        # data_sounds[catid]['PNP'].append(fsid)
        # single vote of PNP may be a bit unreliable. safer to send it to U group
        # thus it goes to LQ (and not LQprior)
        data_sounds[catid]['U'].append(fsid)
        fsids_assigned_cat.append(fsid)
    elif 1.0 not in votes and 0.5 not in votes and -1.0 in votes and 0.0 not in votes:
        data_sounds[catid]['NP'].append(fsid)
        fsids_assigned_cat.append(fsid)
    elif 1.0 not in votes and 0.5 not in votes and -1.0 not in votes and 0.0 in votes:
        data_sounds[catid]['U'].append(fsid)
        fsids_assigned_cat.append(fsid)


    # rest of 11 cases. placing first the PP options in case there is some error.

    # 8 PP and PNP
    elif 1.0 in votes and 0.5 in votes and -1.0 not in votes and 0.0 not in votes:
        data_sounds[catid]['PNP'].append(fsid)
        fsids_assigned_cat.append(fsid)
        #candidate here
        count_risky_PP += 1
    # 9 PP and PNP and U
    elif 1.0 in votes and 0.5 in votes and -1.0 not in votes and 0.0 in votes:
        data_sounds[catid]['PNP'].append(fsid)
        fsids_assigned_cat.append(fsid)
        #candidate here
        count_risky_PP += 1

    # 1: NP and U
    elif 1.0 not in votes and 0.5 not in votes and -1.0 in votes and 0.0 in votes:
        data_sounds[catid]['NP'].append(fsid)
        fsids_assigned_cat.append(fsid)
    # 2: PNP and U
    elif 1.0 not in votes and 0.5 in votes and -1.0 not in votes and 0.0 in votes:
        data_sounds[catid]['U'].append(fsid)
        # candidate here
        fsids_assigned_cat.append(fsid)
    # 3: PNP and NP
    elif 1.0 not in votes and 0.5 in votes and -1.0 in votes and 0.0 not in votes:
        data_sounds[catid]['NP'].append(fsid)
        fsids_assigned_cat.append(fsid)
    # 4: PNP and NP and U
    elif 1.0 not in votes and 0.5 in votes and -1.0 in votes and 0.0 in votes:
        data_sounds[catid]['NP'].append(fsid)
        fsids_assigned_cat.append(fsid)


    # 5: PP and U
    elif 1.0 in votes and 0.5 not in votes and -1.0 not in votes and 0.0 in votes:
        data_sounds[catid]['U'].append(fsid)
        # candidate here
        fsids_assigned_cat.append(fsid)
    # 6: PP and NP
    elif 1.0 in votes and 0.5 not in votes and -1.0 in votes and 0.0 not in votes:
        data_sounds[catid]['U'].append(fsid)
        fsids_assigned_cat.append(fsid)
    # 7: PP and NP and U
    elif 1.0 in votes and 0.5 not in votes and -1.0 in votes and 0.0 in votes:
        data_sounds[catid]['U'].append(fsid)
        fsids_assigned_cat.append(fsid)

    # 10: PP and PNP and NP
    elif 1.0 in votes and 0.5 in votes and -1.0 in votes and 0.0 not in votes:
        data_sounds[catid]['U'].append(fsid)
        fsids_assigned_cat.append(fsid)
    # 11: PP and PNP and NP and U
    elif 1.0 in votes and 0.5 in votes and -1.0 in votes and 0.0 in votes:
        data_sounds[catid]['U'].append(fsid)
        fsids_assigned_cat.append(fsid)

    else:
        # print('\n something unexpetected happened in the mapping********************* \n')
        error_mapping_count_cat += 1
        # sys.exit('something unexpetected happened in the mapping!')

    return data_sounds, fsids_assigned_cat, error_mapping_count_cat, count_risky_PP

In [3]:
# Extract from all the votes only the domestic ones
domestic_votes = dict()
#votes of only the domestic categories (that I control from the external json file i.e. I can add or remove categories)
domestic_votes_jan = {c: data_votes_jan[c] for c in data_votes_jan if c in domestic_ids.keys()}
domestic_votes_mar = {c: data_votes_mar[c] for c in data_votes_mar if c in domestic_ids.keys()}
domestic_votes_may_2 = {c: data_votes_may_2[c] for c in data_votes_may_2 if c in domestic_ids.keys()}
domestic_votes_may_9 = {c: data_votes_may_9[c] for c in data_votes_may_9 if c in domestic_ids.keys()}

In [4]:
# create copy of data_votes
data_sounds_jan = copy.deepcopy(domestic_votes_jan)
for catid, vote_groups in data_sounds_jan.items():
    data_sounds_jan[catid]['PP'] = []
    data_sounds_jan[catid]['PNP'] = []
    data_sounds_jan[catid]['NP'] = []
    data_sounds_jan[catid]['U'] = []
    data_sounds_jan[catid]['QE'] = 0    # initialzed to 0. only if more than MIN_VOTES_CAT, we compute it

In [5]:
# create copy of data_votes
data_sounds_mar = copy.deepcopy(domestic_votes_mar)
for catid, vote_groups in data_sounds_mar.items():
    data_sounds_mar[catid]['PP'] = []
    data_sounds_mar[catid]['PNP'] = []
    data_sounds_mar[catid]['NP'] = []
    data_sounds_mar[catid]['U'] = []
    data_sounds_mar[catid]['QE'] = 0    # initialzed to 0. only if more than MIN_VOTES_CAT, we compute it

In [6]:
# create copy of data_votes
data_sounds_may_2 = copy.deepcopy(domestic_votes_may_2)
for catid, vote_groups in data_sounds_may_2.items():
    data_sounds_may_2[catid]['PP'] = []
    data_sounds_may_2[catid]['PNP'] = []
    data_sounds_may_2[catid]['NP'] = []
    data_sounds_may_2[catid]['U'] = []
    data_sounds_may_2[catid]['QE'] = 0    # initialzed to 0. only if more than MIN_VOTES_CAT, we compute it

In [7]:
# create copy of data_votes
data_sounds_may_9 = copy.deepcopy(domestic_votes_may_9)
for catid, vote_groups in data_sounds_may_9.items():
    data_sounds_may_9[catid]['PP'] = []
    data_sounds_may_9[catid]['PNP'] = []
    data_sounds_may_9[catid]['NP'] = []
    data_sounds_may_9[catid]['U'] = []
    data_sounds_may_9[catid]['QE'] = 0    # initialzed to 0. only if more than MIN_VOTES_CAT, we compute it

In [8]:
for catid, vote_groups in domestic_votes_jan.items():
    # list to keep track of assigned fsids within a category, to achieve disjoint subsets of audio samples
    fsids_assigned_cat = []
    error_mapping_count_cat = 0

    # check GT in PP
    # check GT in the rest of the groups
    # if GT does not exist, take mapping decision without inter-annotator agreement
    for fsid in vote_groups['PP']:
        # print fsid
        # search for GT in this group
        if vote_groups['PP'].count(fsid) > 1:
            if fsid not in fsids_assigned_cat:
                data_sounds_jan[catid]['PP'].append(fsid)
                fsids_assigned_cat.append(fsid)
        else:
            # search for GT in other groups of votes
            data_sounds_jan, fsids_assigned_cat, assigned = check_GT('PNP', fsid, catid, vote_groups, fsids_assigned_cat,
                                                                 data_sounds_jan)
            if not assigned:
                data_sounds_jan, fsids_assigned_cat, assigned = check_GT('U', fsid, catid, vote_groups, fsids_assigned_cat,
                                                                     data_sounds_jan)
            if not assigned:
                data_sounds_jan, fsids_assigned_cat, assigned = check_GT('NP', fsid, catid, vote_groups, fsids_assigned_cat,
                                                                     data_sounds_jan)

        # no GT was found for the annotation (2 votes in the same group).
        # we must take decisions without inter-annotator agreement

        if fsid not in fsids_assigned_cat:
            # map the voted sound to a disjoint group  without inter-annotator agreement
            data_sounds_jan, fsids_assigned_cat, error_mapping_count_cat, count_risky_PP = map_votedsound_2_disjointgroups_wo_agreement(
                fsid, catid, vote_groups, fsids_assigned_cat, data_sounds_jan, error_mapping_count_cat, count_risky_PP)

In [9]:
for catid, vote_groups in domestic_votes_mar.items():
    # list to keep track of assigned fsids within a category, to achieve disjoint subsets of audio samples
    fsids_assigned_cat = []
    error_mapping_count_cat = 0

    # check GT in PP
    # check GT in the rest of the groups
    # if GT does not exist, take mapping decision without inter-annotator agreement
    for fsid in vote_groups['PP']:
        # print fsid
        # search for GT in this group
        if vote_groups['PP'].count(fsid) > 1:
            if fsid not in fsids_assigned_cat:
                data_sounds_mar[catid]['PP'].append(fsid)
                fsids_assigned_cat.append(fsid)
        else:
            # search for GT in other groups of votes
            data_sounds_mar, fsids_assigned_cat, assigned = check_GT('PNP', fsid, catid, vote_groups, fsids_assigned_cat,
                                                                 data_sounds_mar)
            if not assigned:
                data_sounds_mar, fsids_assigned_cat, assigned = check_GT('U', fsid, catid, vote_groups, fsids_assigned_cat,
                                                                     data_sounds_mar)
            if not assigned:
                data_sounds_mar, fsids_assigned_cat, assigned = check_GT('NP', fsid, catid, vote_groups, fsids_assigned_cat,
                                                                     data_sounds_mar)

        # no GT was found for the annotation (2 votes in the same group).
        # we must take decisions without inter-annotator agreement

        if fsid not in fsids_assigned_cat:
            # map the voted sound to a disjoint group  without inter-annotator agreement
            data_sounds_mar, fsids_assigned_cat, error_mapping_count_cat, count_risky_PP = map_votedsound_2_disjointgroups_wo_agreement(
                fsid, catid, vote_groups, fsids_assigned_cat, data_sounds_mar, error_mapping_count_cat, count_risky_PP)

In [10]:
for catid, vote_groups in domestic_votes_may_2.items():
    # list to keep track of assigned fsids within a category, to achieve disjoint subsets of audio samples
    fsids_assigned_cat = []
    error_mapping_count_cat = 0

    # check GT in PP
    # check GT in the rest of the groups
    # if GT does not exist, take mapping decision without inter-annotator agreement
    for fsid in vote_groups['PP']:
        # print fsid
        # search for GT in this group
        if vote_groups['PP'].count(fsid) > 1:
            if fsid not in fsids_assigned_cat:
                data_sounds_may_2[catid]['PP'].append(fsid)
                fsids_assigned_cat.append(fsid)
        else:
            # search for GT in other groups of votes
            data_sounds_may_2, fsids_assigned_cat, assigned = check_GT('PNP', fsid, catid, vote_groups, fsids_assigned_cat,
                                                                 data_sounds_may_2)
            if not assigned:
                data_sounds_may_2, fsids_assigned_cat, assigned = check_GT('U', fsid, catid, vote_groups, fsids_assigned_cat,
                                                                     data_sounds_may_2)
            if not assigned:
                data_sounds_may_2, fsids_assigned_cat, assigned = check_GT('NP', fsid, catid, vote_groups, fsids_assigned_cat,
                                                                     data_sounds_may_2)

        # no GT was found for the annotation (2 votes in the same group).
        # we must take decisions without inter-annotator agreement

        if fsid not in fsids_assigned_cat:
            # map the voted sound to a disjoint group  without inter-annotator agreement
            data_sounds_may_2, fsids_assigned_cat, error_mapping_count_cat, count_risky_PP = map_votedsound_2_disjointgroups_wo_agreement(
                fsid, catid, vote_groups, fsids_assigned_cat, data_sounds_may_2, error_mapping_count_cat, count_risky_PP)

In [11]:
for catid, vote_groups in domestic_votes_may_9.items():
    # list to keep track of assigned fsids within a category, to achieve disjoint subsets of audio samples
    fsids_assigned_cat = []
    error_mapping_count_cat = 0

    # check GT in PP
    # check GT in the rest of the groups
    # if GT does not exist, take mapping decision without inter-annotator agreement
    for fsid in vote_groups['PP']:
        # print fsid
        # search for GT in this group
        if vote_groups['PP'].count(fsid) > 1:
            if fsid not in fsids_assigned_cat:
                data_sounds_may_9[catid]['PP'].append(fsid)
                fsids_assigned_cat.append(fsid)
        else:
            # search for GT in other groups of votes
            data_sounds_may_9, fsids_assigned_cat, assigned = check_GT('PNP', fsid, catid, vote_groups, fsids_assigned_cat,
                                                                 data_sounds_may_9)
            if not assigned:
                data_sounds_may_9, fsids_assigned_cat, assigned = check_GT('U', fsid, catid, vote_groups, fsids_assigned_cat,
                                                                     data_sounds_may_9)
            if not assigned:
                data_sounds_may_9, fsids_assigned_cat, assigned = check_GT('NP', fsid, catid, vote_groups, fsids_assigned_cat,
                                                                     data_sounds_may_9)

        # no GT was found for the annotation (2 votes in the same group).
        # we must take decisions without inter-annotator agreement

        if fsid not in fsids_assigned_cat:
            # map the voted sound to a disjoint group  without inter-annotator agreement
            data_sounds_may_9, fsids_assigned_cat, error_mapping_count_cat, count_risky_PP = map_votedsound_2_disjointgroups_wo_agreement(
                fsid, catid, vote_groups, fsids_assigned_cat, data_sounds_may_9, error_mapping_count_cat, count_risky_PP)

In [12]:
dataset_jan = dict()
for key,value in domestic_ids.items():
    dataset_jan[value] = data_sounds_jan[key]["PP"]
    
dataset_mar = dict()
for key,value in domestic_ids.items():
    dataset_mar[value] = data_sounds_mar[key]["PP"]
    
dataset_may_2 = dict()
for key,value in domestic_ids.items():
    dataset_may_2[value] = data_sounds_may_2[key]["PP"]
    
dataset_may_9 = dict()
for key,value in domestic_ids.items():
    dataset_may_9[value] = data_sounds_may_9[key]["PP"]

In [13]:
counter_jan = 0
counter_mar = 0
counter_may_2 = 0
counter_may_9 = 0

In [14]:
for key,value in dataset_jan.items():
    counter_jan = counter_jan + len(value)

In [15]:
counter_jan

1325

In [16]:
for key,value in dataset_mar.items():
    counter_mar = counter_mar + len(value)

In [17]:
counter_mar

1849

In [18]:
for key,value in dataset_may_2.items():
    counter_may_2 = counter_may_2 + len(value)

In [19]:
counter_may_2

2217

In [20]:
for key,value in dataset_may_9.items():
    counter_may_9 = counter_may_9 + len(value)

In [21]:
counter_may_9

2140

In [22]:
#probably some only pp was annotated with something else

In [23]:
dataset_may_9

{'Bathtub (filling or washing)': [205088,
  333348,
  330555,
  360374,
  67320,
  370012,
  143247,
  104990,
  46645,
  90793,
  360735,
  212111,
  344422,
  265339,
  102687,
  121548,
  126984,
  365653,
  340873,
  85756,
  212108,
  85901,
  206167,
  324773,
  223321,
  144822,
  93958,
  94818,
  168427,
  121541,
  134971,
  346767,
  143619,
  274045],
 'Chopping (food)': [348081,
  87984,
  366620,
  269484,
  201033,
  66679,
  325223,
  215749,
  98334,
  211703,
  272220,
  360788,
  77920,
  211701,
  369234,
  369236,
  201032,
  360802,
  211702,
  53292,
  188192,
  167738,
  365784,
  173953,
  211700,
  117603,
  378689,
  360790,
  344901,
  360649,
  360794,
  237359,
  87506,
  348128,
  327592,
  167514,
  366252,
  360795,
  238207,
  167508,
  369231,
  256288,
  360791,
  87508,
  325936,
  214359,
  125521,
  369235,
  212615,
  338406,
  181612,
  360798,
  211699,
  167516,
  196562],
 'Blender': [46750,
  178493,
  198017,
  46749,
  353530,
  46747,
  9